In [1]:
import geopandas as gpd
import pandas as pd
import vaex

In [2]:
gdf_est_alimentares = gpd.read_file('results/estabelecimentos_alimentares_BR_all.gpkg')

In [3]:
gdf_est_alimentares.shape

(2425121, 7)

In [4]:
df_estabelecimentos = vaex.open('data/brasil_cnpj.hdf5')
df_municipios = vaex.read_csv('data/Municipios.zip',
                    compression='zip', 
                    sep=';', 
                    encoding_errors='replace',
                    names=['cd_municipio', 'nome_municipio'])
df_municipios['cd_municipio_str'] = df_municipios.cd_municipio.to_string().str.pad(4, side='left', fillchar='0')
df_estabelecimentos = df_estabelecimentos.join(df_municipios, left_on='logradouro_cod_municipio', right_on='cd_municipio_str')

In [5]:
df_estabelecimentos.shape

(55813191, 33)

In [6]:
gdf_est_alimentares.shape

(2425121, 7)

In [7]:
df_estabelecimentos['cnpj'] = df_estabelecimentos.cnpj_basico + '/' + df_estabelecimentos.cnpj_ordem + '-' + df_estabelecimentos.cnpj_dv

In [9]:
CNAES_alimentacao = [
    "4711301",
    "4711302",
    "4712100",
    "4721102",
    "4721103",
    "4721104",
    "4722901",
    "4722902",
    "4724500",
    "4729602",
    "4729699",
    "5611201",
    "5611202",
    "5611203",
    "5612100",
    "5620103",
    "5620104"
] 

In [10]:
df_estabelecimentos = df_estabelecimentos[df_estabelecimentos.cnae_principal.isin(CNAES_alimentacao)]
ativos = (df_estabelecimentos.situacao_cadastral == '02')
df_estabelecimentos = df_estabelecimentos[ativos]

In [11]:
df_estabelecimentos.shape

(2440648, 34)

In [12]:
df_estabelecimentos_pandas = df_estabelecimentos.to_pandas_df()

In [14]:
df_estabelecimentos_pandas.dtypes

cnpj_basico                  object
cnpj_ordem                   object
cnpj_dv                      object
matriz_filial                object
nome_fantasia                object
situacao_cadastral           object
data_situacao_cadastral      object
motivo_situacao_cadastral    object
nome_cidade_no_exterior      object
pais                         object
data_inicio_atividade        object
cnae_principal               object
cnae_secundario              object
logradouro_tipo              object
logradouro_nome              object
logradouro_numero            object
logradouro_complemento       object
logradouro_bairro            object
logradouro_cep               object
logradouro_uf                object
logradouro_cod_municipio     object
ddd1                         object
telefone1                    object
ddd2                         object
telefone2                    object
ddd_fax                      object
fax                          object
email                       

In [15]:
gdf_est_alimentares.dtypes

cnpj                 object
long                float64
lat                 float64
confidence          float64
cnae_principal       object
cnae_secundario      object
geometry           geometry
dtype: object

In [13]:
gdf_est_alimentares.join(df_estabelecimentos_pandas, how='left', on='cnpj')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat